In [ ]:
# make connection to google drive to access data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
# set up directory paths to image files
train_path = "/content/drive/My Drive/Deep Learning and Generative AI/breast_cancer/Dataset_BUSI_with_GT"

In [ ]:
save_path_train = "/content/drive/My Drive/Deep Learning and Generative AI/breast_cancer/Dataset_BUSI_with_GT/generated_data_breast"

In [ ]:
# import relevant libraries for analysis
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# import relevant libraries for loading GAN model and analysis
from tensorflow.keras.models import load_model
import tensorflow as tf

In [ ]:
# upload the generator model from Google Drive
gen_path = "/content/drive/My Drive/Deep Learning and Generative AI/breast_cancer/saved_models/generator_model_breast.keras"
gen = load_model(gen_path)

In [ ]:
codings_size = 250

In [41]:
# analyze difference in images for training cancer and normal images
print('(Train) Normal Images #: ',len([file for file in os.listdir(os.path.join(train_path,'normal')) if 'mask' not in file]))
print('(Train) Cancer Images #: ',len([file for file in os.listdir(os.path.join(train_path,'malignant')) if 'mask' not in file]))
print('(Train) To Generate   #: ',len([file for file in os.listdir(os.path.join(train_path,'malignant')) if 'mask' not in file])-len([file for file in os.listdir(os.path.join(train_path,'normal')) if 'mask' not in file]))

(Train) Normal Images #:  133
(Train) Cancer Images #:  210
(Train) To Generate   #:  77


In [ ]:
new_imgs_train = len([file for file in os.listdir(os.path.join(train_path,'malignant')) if 'mask' not in file])-len([file for file in os.listdir(os.path.join(train_path,'normal')) if 'mask' not in file]) # holds number training images to generate

In [ ]:
# make the directories if they don't already exist
if not os.path.exists(save_path_train):
    os.makedirs(save_path_train)

In [ ]:
# generate the number of training images for class imbalance
for i in range(new_imgs_train):
    noise = tf.random.normal(shape=[1, codings_size])
    images = gen(noise)
    plt.imshow(images.numpy().reshape(224, 224, 1), cmap='gray')
    plt.axis('off')
    image_file_path = os.path.join(save_path_train, f'image_{i+1}.png')
    plt.savefig(image_file_path, bbox_inches='tight', pad_inches=0)
    plt.close()

print(f'Saved {new_imgs_train} images to {save_path_train}')

Saved 77 images to /content/drive/My Drive/Deep Learning and Generative AI/breast_cancer/Dataset_BUSI_with_GT/generated_data_breast


In [ ]:
import shutil

# paths to image directories
generated_path = "/content/drive/My Drive/Deep Learning and Generative AI/breast_cancer/Dataset_BUSI_with_GT/generated_data_breast"

# create new directory to hold both sets of images
combined_path = "/content/drive/My Drive/Deep Learning and Generative AI/breast_cancer/Dataset_BUSI_with_GT/combined_images_train"
if not os.path.exists(combined_path):
    os.makedirs(combined_path)

# class name for generated images
subdir = 'normal'

# create links to images in both directories
src_dir_train = os.path.join(train_path, subdir)
src_dir_generated = generated_path
dest_dir = os.path.join(combined_path, subdir)

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

# copy images from both directories to the combined directory
for src_dir in [os.path.join(train_path, subdir), generated_path]:
    for img in os.listdir(src_dir):
        src_img_path = os.path.join(src_dir, img)
        dest_img_path = os.path.join(dest_dir, img)
        if not os.path.exists(dest_img_path):
            shutil.copy(src_img_path, dest_img_path)